# Lysozyme + Mutations tutorial using BIOBB
Based on the oficial Gromacs tutorial: http://www.mdtutorials.com/gmx/lysozyme/01_pdb2gmx.html

In [1]:
import os
import sys
import nglview
import ipywidgets

## Dowload the initial Structure
Download the initial PDB file from a PDB code (1AKI) from the IRB (http://mmb.irbbarcelona.org/api) REST API.
The PDB will be dowloaded without solvent or HETATM.

In [2]:
#Download PDB
#Import module
from biobb_io.mmb_api.pdb import MmbPdb
# Create prop dict and inputs/outputs
prop = {'pdb_code':'1aki'}
downloaded_pdb_path = '1aki.pdb'
#Create and launch bb
MmbPdb(output_pdb_path=downloaded_pdb_path, properties=prop).launch()

2019-01-21 22:27:49,422 [MainThread  ] [INFO ]  Downloading: 1aki from: https://files.rcsb.org/download/1aki.pdb
2019-01-21 22:27:50,087 [MainThread  ] [INFO ]  Writting pdb to: /Users/pau/projects/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki.pdb
2019-01-21 22:27:50,088 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


In [3]:
#Show protein
view = nglview.show_file(downloaded_pdb_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

## Modeling the missing heavy atoms in the structure side chains
This BB will reconstruct missing atoms from residue side chains and detect all kinds of clashes

In [4]:
#Check & Fix PDB
#Import module
from biobb_model.model.fix_side_chain import FixSideChain
# Create prop dict and inputs/outputs
fixed_pdb_path = '1aki_fixed.pdb'
#Create and launch bb
FixSideChain(input_pdb_path=downloaded_pdb_path, output_pdb_path=fixed_pdb_path).launch()

In [5]:
#Show protein
view = nglview.show_file(fixed_pdb_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

## Modeling mutation list
This BB will mutate existing residues to the requested ones.

In [6]:
#Mutate protein
# Import module to test
from biobb_model.model.mutate import Mutate
# Create prop dict and inputs/outputs
prop = {'mutation_list': 'Val2Tyr'}
mutated_pdb_path = '1aki_mutated.pdb'
#Create and launch bb
Mutate(input_pdb_path=fixed_pdb_path,output_pdb_path=mutated_pdb_path, properties=prop).launch()

In [7]:
#Show log and protein
view1 = nglview.show_file(fixed_pdb_path)
view1.add_representation(repr_type='ball+stick', selection='2:A')
view1._remote_call('setSize', target='Widget', args=['500px','600px'])
view1.camera='orthographic'
view2 = nglview.show_file(mutated_pdb_path)
view2.add_representation(repr_type='ball+stick', selection='2:A')
view2._remote_call('setSize', target='Widget', args=['500px','600px'])
view2.camera='orthographic'
ipywidgets.HBox([view1, view2])

## Generate the topology
Generate the topology using [Pdb2gmx](http://manual.gromacs.org/programs/gmx-pdb2gmx.html) module.
The default forcefield is [amber99sb-ildn](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2970904/) and the default water_type is [spce](http://www.sklogwiki.org/SklogWiki/index.php/SPC/E_model_of_water).
This BB will generate 2 main files:
-  A compressed ZIP file containing:
> -  The [Gromacs topology file](http://manual.gromacs.org/online/top.html) (.top)
> -  The [Gromacs position restraint file](http://manual.gromacs.org/online/itp.html) (.itp)
-  A post-processed [Gromacs structure file](http://manual.gromacs.org/current/online/gro.html) (.gro)

In [8]:
#Create system topology
# Import module
from biobb_md.gromacs.pdb2gmx import Pdb2gmx
# Create inputs/outputs
output_pdb2gmx_gro_path = '1aki_pdb2gmx.gro'
output_pdb2gmx_top_zip_path = '1aki_pdb2gmx_top.zip'
#Create and launch bb
Pdb2gmx(input_pdb_path=mutated_pdb_path, output_gro_path=output_pdb2gmx_gro_path, output_top_zip_path=output_pdb2gmx_top_zip_path).launch()

2019-01-21 22:27:56,566 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20181 version detected
2019-01-21 22:27:56,889 [MainThread  ] [INFO ]  gmx pdb2gmx -f 1aki_mutated.pdb -o 1aki_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i p2g.itp

2019-01-21 22:27:56,891 [MainThread  ] [INFO ]  Exit code 0

2019-01-21 22:27:56,892 [MainThread  ] [INFO ]  
Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

Reading 1aki_mutated.pdb...
Read 1006 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 1 chains and 0 blocks of water and 129 residues with 1006 atoms

  chain  #res #atoms
  1 'A'   129   1006  

Reading residue database... (amber99sb-ildn)
Processing chain 1 'A' (1006 atoms, 129 residues)
Identified residue LYS1 as a starting terminus.
Identified residue LEU129 as a ending terminus.
Checking for duplicate atoms....
Generating any missing hydrogen atoms and/or adding termini.
Now there are 129 residues with 1965 atoms


0

In [9]:
#Show protein
view = nglview.show_file(output_pdb2gmx_gro_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

Note that hydrogen atoms had been added to the structure.
***

## Create the solvent box
Create the solvent box using the [Editconf](http://manual.gromacs.org/programs/gmx-editconf.html) module. The box will be centered, the distance between the solute and the box will be 1.0nm and the box shape will be cubic by default. The main output of this BB will be an updated post-procesed Gromacs structure file (.gro). 

In [10]:
# Editconf: Create solvent box
# Import module
from biobb_md.gromacs.editconf import Editconf
# Create prop dict and inputs/outputs
output_editconf_gro_path = '1aki_editconf.gro'
#Create and launch bb
Editconf(input_gro_path=output_pdb2gmx_gro_path, output_gro_path=output_editconf_gro_path).launch()

2019-01-21 22:28:00,294 [MainThread  ] [INFO ]  GROMACS Editconf 20181 version detected
2019-01-21 22:28:00,296 [MainThread  ] [INFO ]  Centering molecule in the box.
2019-01-21 22:28:00,297 [MainThread  ] [INFO ]  Distance of the box to molecule:   1.00
2019-01-21 22:28:00,297 [MainThread  ] [INFO ]  Box type: cubic
2019-01-21 22:28:00,320 [MainThread  ] [INFO ]  gmx editconf -f 1aki_pdb2gmx.gro -o 1aki_editconf.gro -d 1.0 -bt cubic -c

2019-01-21 22:28:00,322 [MainThread  ] [INFO ]  Exit code 0

2019-01-21 22:28:00,323 [MainThread  ] [INFO ]  Read 1965 atoms
Volume: 55.8378 nm^3, corresponds to roughly 25100 electrons
No velocities found
    system size :  3.817  4.234  3.454 (nm)
    diameter    :  5.010               (nm)
    center      :  2.781  2.484  0.015 (nm)
    box vectors :  3.817  4.235  3.455 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :  55.84               (nm^3)
    shift       :  0.724  1.021  3.490 (nm)
new center      :  3.505  3.505  3.50

0

## Fill the solvent box with water molecules
Fill the solvent box using the Gromacs Solvate module. The "spc216.gro" will be default solvent model. The main output of this BB will be an updated post-procesed Gromacs structure file (.gro) and a zip file containing the updated topology file (.top) and the restriction files (.itp).

In [11]:
# Solvate: Fill the box with water molecules
from biobb_md.gromacs.solvate import Solvate
# Create prop dict and inputs/outputs
output_solvate_gro_path = '1aki_solvate.gro'
output_solvate_top_zip_path = '1aki_solvate_top.zip'
#Create and launch bb
Solvate(input_solute_gro_path=output_editconf_gro_path, output_gro_path=output_solvate_gro_path, input_top_zip_path=output_pdb2gmx_top_zip_path, output_top_zip_path=output_solvate_top_zip_path).launch()

2019-01-21 22:28:02,139 [MainThread  ] [INFO ]  GROMACS Solvate 20181 version detected
2019-01-21 22:28:02,146 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_pdb2gmx_top.zip
2019-01-21 22:28:02,146 [MainThread  ] [INFO ]  to:
2019-01-21 22:28:02,147 [MainThread  ] [INFO ]  ['9dabab49-f15e-4e6d-a825-70bcc7756e39/p2g.itp', '9dabab49-f15e-4e6d-a825-70bcc7756e39/p2g.top']
2019-01-21 22:28:02,148 [MainThread  ] [INFO ]  Unzipping: 
2019-01-21 22:28:02,150 [MainThread  ] [INFO ]  1aki_pdb2gmx_top.zip
2019-01-21 22:28:02,151 [MainThread  ] [INFO ]  To: 
2019-01-21 22:28:02,152 [MainThread  ] [INFO ]  9dabab49-f15e-4e6d-a825-70bcc7756e39/p2g.itp
2019-01-21 22:28:02,153 [MainThread  ] [INFO ]  9dabab49-f15e-4e6d-a825-70bcc7756e39/p2g.top
2019-01-21 22:28:02,373 [MainThread  ] [INFO ]  gmx solvate -cp 1aki_editconf.gro -cs spc216.gro -o 1aki_solvate.gro -p 9dabab49-f15e-4e6d-a825-70bcc7756e39/p2g.top

2019-01-21 22:28:02,3

0

In [12]:
#Show protein
view = nglview.show_file(output_solvate_gro_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='green')
view.add_representation(repr_type='ball+stick', selection='SOL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

## Preprocess ion generation
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for ion generation, using the Grompp module. The main default parameters for this execution are:
-  integrator  = steep         ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01          ; Minimization step size
-  nsteps      = 50000         ; Maximum number of (minimization) steps to perform

The main output of this BB will be the portable binary run file (.tpr).

In [13]:
# Grompp: Creating portable binary run file for ion generation
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'minimization', 'nsteps':'50000'}}
output_gppion_tpr_path = '1aki_gppion.tpr'
#Create and launch bb
Grompp(input_gro_path=output_solvate_gro_path, input_top_zip_path=output_solvate_top_zip_path, output_tpr_path=output_gppion_tpr_path,  properties=prop).launch()

2019-01-21 22:28:16,718 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-01-21 22:28:16,720 [MainThread  ] [INFO ]  Will run a minimization md of 50000 steps
2019-01-21 22:28:16,726 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_solvate_top.zip
2019-01-21 22:28:16,727 [MainThread  ] [INFO ]  to:
2019-01-21 22:28:16,727 [MainThread  ] [INFO ]  ['45e2fdac-cd54-46e9-889d-71f2add4a710/p2g.itp', '45e2fdac-cd54-46e9-889d-71f2add4a710/p2g.top']
2019-01-21 22:28:16,728 [MainThread  ] [INFO ]  Unzipping: 
2019-01-21 22:28:16,729 [MainThread  ] [INFO ]  1aki_solvate_top.zip
2019-01-21 22:28:16,730 [MainThread  ] [INFO ]  To: 
2019-01-21 22:28:16,730 [MainThread  ] [INFO ]  45e2fdac-cd54-46e9-889d-71f2add4a710/p2g.itp
2019-01-21 22:28:16,731 [MainThread  ] [INFO ]  45e2fdac-cd54-46e9-889d-71f2add4a710/p2g.top
2019-01-21 22:28:16,998 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_solvate.gro -r 1a

0

## Ion generation
Replace solvent molecules to neutralice the system and then reach a 0.05 mol/litre concentration by default. Using the Genion module.
The main output of this BB will be an updated post-procesed Gromacs structure file (.gro) and a zip file containing the updated topology file (.top) and the restriction files (.itp).

In [14]:
# Genion: Adding ions to reach a 0.05 nm concentration
from biobb_md.gromacs.genion import Genion
# Create prop dict and inputs/outputs
prop={'neutral':True, 'concentration':0.05}
output_genion_gro_path = '1aki_genion.gro'
output_genion_top_zip_path = '1aki_genion_top.zip'
#Create and launch bb
Genion(input_tpr_path=output_gppion_tpr_path, output_gro_path=output_genion_gro_path, input_top_zip_path=output_solvate_top_zip_path, output_top_zip_path=output_genion_top_zip_path, properties=prop).launch()

2019-01-21 22:28:23,887 [MainThread  ] [INFO ]  GROMACS Genion 20181 version detected
2019-01-21 22:28:23,888 [MainThread  ] [INFO ]  To reach up 0.05 mol/litre concentration
2019-01-21 22:28:23,894 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_solvate_top.zip
2019-01-21 22:28:23,895 [MainThread  ] [INFO ]  to:
2019-01-21 22:28:23,896 [MainThread  ] [INFO ]  ['5bca5516-d56f-444d-bfb4-c2977e682c44/p2g.itp', '5bca5516-d56f-444d-bfb4-c2977e682c44/p2g.top']
2019-01-21 22:28:23,897 [MainThread  ] [INFO ]  Unzipping: 
2019-01-21 22:28:23,898 [MainThread  ] [INFO ]  1aki_solvate_top.zip
2019-01-21 22:28:23,898 [MainThread  ] [INFO ]  To: 
2019-01-21 22:28:23,899 [MainThread  ] [INFO ]  5bca5516-d56f-444d-bfb4-c2977e682c44/p2g.itp
2019-01-21 22:28:23,900 [MainThread  ] [INFO ]  5bca5516-d56f-444d-bfb4-c2977e682c44/p2g.top
2019-01-21 22:28:24,011 [MainThread  ] [INFO ]  echo "SOL" | gmx genion -s 1aki_gppion.tpr -o 1aki_

0

In [15]:
#Show protein
view = nglview.show_file(output_genion_gro_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

## Preprocess energy minimization
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for energy minimization, using the Grompp module. The main default parameters for this execution are:
-  integrator  = steep         ; Algorithm (steep = steepest descent minimization)
-  emtol       = 500.0         ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01          ; Minimization step size
-  nsteps      = 50000         ; Maximum number of (minimization) steps to perform

The main output of this BB will be the portable binary run file (.tpr).

In [16]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'minimization', 'nsteps':'50000', 'emtol':'500'}}
output_gppmin_tpr_path = '1aki_gppmin.tpr'
#Create and launch bb
Grompp(input_gro_path=output_genion_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppmin_tpr_path,  properties=prop).launch()

2019-01-21 22:28:29,176 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-01-21 22:28:29,179 [MainThread  ] [INFO ]  Will run a minimization md of 50000 steps
2019-01-21 22:28:29,185 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_genion_top.zip
2019-01-21 22:28:29,185 [MainThread  ] [INFO ]  to:
2019-01-21 22:28:29,186 [MainThread  ] [INFO ]  ['1e8a6a8c-7346-4af9-8025-c77f144e2277/p2g.itp', '1e8a6a8c-7346-4af9-8025-c77f144e2277/p2g.top']
2019-01-21 22:28:29,187 [MainThread  ] [INFO ]  Unzipping: 
2019-01-21 22:28:29,188 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-01-21 22:28:29,189 [MainThread  ] [INFO ]  To: 
2019-01-21 22:28:29,190 [MainThread  ] [INFO ]  1e8a6a8c-7346-4af9-8025-c77f144e2277/p2g.itp
2019-01-21 22:28:29,191 [MainThread  ] [INFO ]  1e8a6a8c-7346-4af9-8025-c77f144e2277/p2g.top
2019-01-21 22:28:29,455 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_genion.gro -r 1aki_

0

## Execute system equilibration
Execute the energy minimization using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [17]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_min_trr_path = '1aki_min.trr'
output_min_gro_path = '1aki_min.gro'
output_min_edr_path = '1aki_min.edr'
output_min_log_path = '1aki_min.log'
#Create and launch bb
Mdrun(input_tpr_path=output_gppmin_tpr_path, output_trr_path=output_min_trr_path, output_gro_path=output_min_gro_path, output_edr_path=output_min_edr_path, output_log_path=output_min_log_path).launch()

2019-01-21 22:28:31,864 [MainThread  ] [INFO ]  GROMACS Mdrun 20181 version detected
2019-01-21 22:29:15,778 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppmin.tpr -o 1aki_min.trr -c 1aki_min.gro -e 1aki_min.edr -g 1aki_min.log -nt 0

2019-01-21 22:29:15,780 [MainThread  ] [INFO ]  Exit code 0

2019-01-21 22:29:15,781 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Mi

0

## Preprocess system temperature equilibration
Equilibrate the solvent and ions restraining the protein heavy atoms
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for system equilibration, using the Grompp module. The main default parameters for this execution are:
-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl                   = no
-  gen_vel                  = yes
-  gen_temp                 = 300
-  gen_seed                 = -1

The main output of this BB will be the portable binary run file (.tpr).

In [18]:
# Grompp: Creating portable binary run file for Equilibration
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'nvt', 'nsteps':'5000'}}
output_gppnvt_tpr_path = '1aki_gppnvt.tpr'
#Create and launch bb
Grompp(input_gro_path=output_min_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppnvt_tpr_path,  properties=prop).launch()

2019-01-21 22:29:38,613 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-01-21 22:29:38,616 [MainThread  ] [INFO ]  Will run a nvt md of 5000 steps
2019-01-21 22:29:38,621 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_genion_top.zip
2019-01-21 22:29:38,623 [MainThread  ] [INFO ]  to:
2019-01-21 22:29:38,623 [MainThread  ] [INFO ]  ['fef95ebf-dad8-4bf9-81f5-41d5f8f9342e/p2g.itp', 'fef95ebf-dad8-4bf9-81f5-41d5f8f9342e/p2g.top']
2019-01-21 22:29:38,624 [MainThread  ] [INFO ]  Unzipping: 
2019-01-21 22:29:38,625 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-01-21 22:29:38,626 [MainThread  ] [INFO ]  To: 
2019-01-21 22:29:38,627 [MainThread  ] [INFO ]  fef95ebf-dad8-4bf9-81f5-41d5f8f9342e/p2g.itp
2019-01-21 22:29:38,628 [MainThread  ] [INFO ]  fef95ebf-dad8-4bf9-81f5-41d5f8f9342e/p2g.top
2019-01-21 22:29:39,022 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_min.gro -r 1aki_min.gro -p fe

0

## Execute system temperature equilibration
Execute the system equilibration using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [19]:
# Mdrun: Running Equilibration NVT
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_nvt_trr_path = '1aki_nvt.trr'
output_nvt_gro_path = '1aki_nvt.gro'
output_nvt_edr_path = '1aki_nvt.edr'
output_nvt_log_path = '1aki_nvt.log'
output_nvt_cpt_path = '1aki_nvt.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppnvt_tpr_path, output_trr_path=output_nvt_trr_path, output_gro_path=output_nvt_gro_path, output_edr_path=output_nvt_edr_path, output_log_path=output_nvt_log_path, output_cpt_path=output_nvt_cpt_path).launch()

2019-01-21 22:29:41,838 [MainThread  ] [INFO ]  GROMACS Mdrun 20181 version detected
2019-01-21 22:30:25,884 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppnvt.tpr -o 1aki_nvt.trr -c 1aki_nvt.gro -e 1aki_nvt.edr -g 1aki_nvt.log -nt 0 -cpo 1aki_nvt.cpt

2019-01-21 22:30:25,885 [MainThread  ] [INFO ]  Exit code 0

2019-01-21 22:30:25,886 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexe

0

## Preprocess system pressure equilibration
Equilibrate the solvent and ions restraining the protein heavy atoms.
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for system equilibration, using the Grompp module. The main default parameters for this execution are:
-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

The main output of this BB will be the portable binary run file (.tpr).

In [20]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'npt', 'nsteps':'5000'}}
output_gppnpt_tpr_path = '1aki_gppnpt.tpr'
#Create and launch bb
Grompp(input_gro_path=output_nvt_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppnpt_tpr_path, input_cpt_path=output_nvt_cpt_path,  properties=prop).launch()

2019-01-21 22:30:30,844 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-01-21 22:30:30,847 [MainThread  ] [INFO ]  Will run a npt md of 5000 steps
2019-01-21 22:30:30,853 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_genion_top.zip
2019-01-21 22:30:30,853 [MainThread  ] [INFO ]  to:
2019-01-21 22:30:30,855 [MainThread  ] [INFO ]  ['16639d0a-6377-4a92-9912-5f5419bba387/p2g.itp', '16639d0a-6377-4a92-9912-5f5419bba387/p2g.top']
2019-01-21 22:30:30,855 [MainThread  ] [INFO ]  Unzipping: 
2019-01-21 22:30:30,856 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-01-21 22:30:30,857 [MainThread  ] [INFO ]  To: 
2019-01-21 22:30:30,858 [MainThread  ] [INFO ]  16639d0a-6377-4a92-9912-5f5419bba387/p2g.itp
2019-01-21 22:30:30,859 [MainThread  ] [INFO ]  16639d0a-6377-4a92-9912-5f5419bba387/p2g.top
2019-01-21 22:30:31,294 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_nvt.gro -r 1aki_nvt.gro -p 16

0

## Execute system pressure equilibration
Execute the system equilibration using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [21]:
# Mdrun: Running minimization NPT
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_npt_trr_path = '1aki_npt.trr'
output_npt_gro_path = '1aki_npt.gro'
output_npt_edr_path = '1aki_npt.edr'
output_npt_log_path = '1aki_npt.log'
output_npt_cpt_path = '1aki_npt.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppnpt_tpr_path, output_trr_path=output_npt_trr_path, output_gro_path=output_npt_gro_path, output_edr_path=output_npt_edr_path, output_log_path=output_npt_log_path, output_cpt_path=output_npt_cpt_path).launch()

2019-01-21 22:30:33,861 [MainThread  ] [INFO ]  GROMACS Mdrun 20181 version detected
2019-01-21 22:31:19,404 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppnpt.tpr -o 1aki_npt.trr -c 1aki_npt.gro -e 1aki_npt.edr -g 1aki_npt.log -nt 0 -cpo 1aki_npt.cpt

2019-01-21 22:31:19,406 [MainThread  ] [INFO ]  Exit code 0

2019-01-21 22:31:19,407 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexe

0

## Preprocess free dynamics
Free molecular dynamics simulation.
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for free molecular dynamics simulation, using the Grompp module. The main default parameters for this execution are:
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000

The main output of this BB will be the portable binary run file (.tpr).

In [29]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'free', 'nsteps':'15000'}}
output_gppmd_tpr_path = '1aki_gppmd.tpr'
#Create and launch bb
Grompp(input_gro_path=output_npt_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppmd_tpr_path, input_cpt_path=output_npt_cpt_path, properties=prop).launch()

2019-01-21 22:35:01,663 [MainThread  ] [INFO ]  GROMACS Grompp 20181 version detected
2019-01-21 22:35:01,666 [MainThread  ] [INFO ]  Will run a free md of 30.0 pico seconds
2019-01-21 22:35:01,671 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_genion_top.zip
2019-01-21 22:35:01,672 [MainThread  ] [INFO ]  to:
2019-01-21 22:35:01,673 [MainThread  ] [INFO ]  ['8448a39c-a4bd-4233-b74d-90e8a0e1c1d9/p2g.itp', '8448a39c-a4bd-4233-b74d-90e8a0e1c1d9/p2g.top']
2019-01-21 22:35:01,674 [MainThread  ] [INFO ]  Unzipping: 
2019-01-21 22:35:01,675 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-01-21 22:35:01,676 [MainThread  ] [INFO ]  To: 
2019-01-21 22:35:01,677 [MainThread  ] [INFO ]  8448a39c-a4bd-4233-b74d-90e8a0e1c1d9/p2g.itp
2019-01-21 22:35:01,678 [MainThread  ] [INFO ]  8448a39c-a4bd-4233-b74d-90e8a0e1c1d9/p2g.top
2019-01-21 22:35:02,020 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_npt.gro -r 1aki_npt.g

0

## Execute free molecular dynamics simulation
Execute the free molecular dynamics simulation  using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [30]:
# Mdrun: Running free dynamics
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_md_trr_path = '1aki_md.trr'
output_md_gro_path = '1aki_md.gro'
output_md_edr_path = '1aki_md.edr'
output_md_log_path = '1aki_md.log'
output_md_cpt_path = '1aki_md.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppmd_tpr_path, output_trr_path=output_md_trr_path, output_gro_path=output_md_gro_path, output_edr_path=output_md_edr_path, output_log_path=output_md_log_path, output_cpt_path=output_md_cpt_path).launch()

2019-01-21 22:35:14,776 [MainThread  ] [INFO ]  GROMACS Mdrun 20181 version detected
2019-01-21 22:37:31,745 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppmd.tpr -o 1aki_md.trr -c 1aki_md.gro -e 1aki_md.edr -g 1aki_md.log -nt 0 -cpo 1aki_md.cpt

2019-01-21 22:37:31,747 [MainThread  ] [INFO ]  Exit code 0

2019-01-21 22:37:31,748 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shve

0

In [31]:
#Show protein
nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_md_trr_path, output_md_gro_path), gui=True)

NGLWidget(count=4)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Clean the directory Remove everything
!find . ! \( -iname "*.ipynb" -o -iname "box.png" \) -exec rm -rf {} 2>/dev/null \; -print